In [65]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
with open("headings.csv","r") as file:
    heading = file.readline()
    heading = heading.split(",")
    #print(heading)
dataset = pd.read_csv("dataset.csv",names=heading)
dataset.head()

,id,title,authors,abstract,subjects,class,is_pdf,is_source,pub_date,update_date
0,1809.03478v1,Towards a Fatality-Aware Benchmark of Probabil...,"[""Wei Zhan"", ""Liting Sun"", ""Yeping Hu"", ""Jiach...",Autonomous vehicles should be able to generate...,"[""cs.RO"", ""cs.AI""]",cs.AI,NaN,NaN,2018-09-10 17:48:58,2018-09-10 17:48:58
1,1809.03470v1,ViZDoom Competitions: Playing Doom from Pixels,"[""Marek Wydmuch"", ""Micha\u0142 Kempka"", ""Wojci...",This paper presents the first two editions of ...,"[""cs.AI"", ""cs.CV"", ""cs.LG"", ""stat.ML""]",cs.AI,NaN,NaN,2018-09-10 17:41:39,2018-09-10 17:41:39
2,1809.03449v1,Exploring Machine Reading Comprehension with E...,"[""Chao Wang"", ""Hui Jiang""]",To apply general knowledge to machine reading ...,"[""cs.AI""]",cs.AI,NaN,NaN,2018-09-10 16:42:22,2018-09-10 16:42:22
3,1809.03428v1,Not Just Privacy: Improving Performance of Pri...,"[""Ji Wang"", ""Jianguo Zhang"", ""Weidong Bao"", ""X...",The increasing demand for on-device deep learn...,"[""cs.LG"", ""cs.AI"", ""cs.DC"", ""stat.ML""]",cs.AI,NaN,NaN,2018-09-10 16:09:58,2018-09-10 16:09:58
4,1809.03406v1,Keep it stupid simple,"[""Erik J Peterson"", ""Necati Alp M\u00fcyesser""...",Deep reinforcement learning can match and exce...,"[""cs.AI""]",cs.AI,NaN,NaN,2018-09-10 15:43:57,2018-09-10 15:43:57


In [4]:
# extract features from abstract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# use english stop_words
stop_words =  list(stopwords.words('english'))

# make a function to split words and choose only alphabetic and words that are not in stop words list
def alphabetic_only(x_string):
    tokens = word_tokenize(x_string.lower())
    # remove all tokens that are not alphabetic
    words = " ".join([word for word in tokens if word.isalpha() and word not in stop_words and len(word)>3])
    return words

def alphabetic_only_l(x_string):
    tokens = word_tokenize(x_string.lower())
    # remove all tokens that are not alphabetic
    words = [word for word in tokens if word.isalpha() and word not in stop_words and len(word)>3]
    return words

In [5]:
abstract_clean = dataset.abstract.apply(lambda my_str: alphabetic_only(my_str))

In [6]:
abstract_clean.head()

0    autonomous vehicles able generate accurate pro...
1    paper presents first editions visual doom comp...
2    apply general knowledge machine reading compre...
3    increasing demand deep learning services calls...
4    deep reinforcement learning match exceed human...
Name: abstract, dtype: object

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# decrease to only using 10thousands feature
no_features = 10000
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tfidf = tfidf_vectorizer.fit_transform(abstract_clean)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features)
tf = tf_vectorizer.fit_transform(abstract_clean)
tf_feature_names = tf_vectorizer.get_feature_names()

/Volumes/HD-500GB/Users/nikolausn/Applications/anaconda/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [27]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

# set number of topics
# breakdown into two times of the classes
num_topics = 80

lda = LatentDirichletAllocation(n_topics=num_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [34]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
#display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)

Topic 0:
heap spreadsheet black evacuation overload organisations scripting contributors hole obligations
Topic 1:
derivatives poisson derivative directional uniqueness discontinuous nonconvex enormous krylov jacobian
Topic 2:
graph graphs tensor matching hierarchy planar decomposition directed bipartite degree
Topic 3:
image images object visual reconstruction color scene objects method information
Topic 4:
programs logic finite formal automata programming synthesis specification checking approach
Topic 5:
communication design protocol using power circuit circuits devices device protocols
Topic 6:
apps statistically bibliometric investigations epidemics career women standardization artists forensics
Topic 7:
sensor motion planning vehicles sensors safety autonomous vehicle system trajectory
Topic 8:
data learning methods deep using method training neural performance classification
Topic 9:
state sparse states policy measurements stochastic localization probability random transition
To

In [9]:
abstract_clean_l = dataset.abstract.apply(lambda my_str: alphabetic_only_l(my_str))
from gensim import corpora
dictionary = corpora.Dictionary(abstract_clean_l)
corpus = [dictionary.doc2bow(text) for text in abstract_clean_l]

In [37]:
import gensim
NUM_TOPICS = 80
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

In [38]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [41]:
"""
import pickle
with open("lda_model.pickle","wb") as checkpoint:
    pickle.dumps((lda,ldamodel))
"""

'\nimport pickle\nwith open("lda_model.pickle","wb") as checkpoint:\n    pickle.dumps((lda,ldamodel))\n'

In [46]:
from hlda.sampler import HierarchicalLDA


In [47]:
n_samples = 500       # no of iterations for the sampler
alpha = 10.0          # smoothing over level distributions
gamma = 1.0           # CRP smoothing parameter; number of imaginary customers at next, as yet unused table
eta = 0.1             # smoothing over topic-word distributions
num_levels = 3        # the number of levels in the tree
display_topics = 50   # the number of iterations between printing a brief summary of the topics so far
n_words = 5           # the number of most probable words to print for each topic after model estimation
with_weights = False  # whether to print the words with the weights

In [11]:
new_corpus=[]
new_vocab=[ x[1] for x in sorted(list(dictionary.items()),key=lambda x:x[0]) ]

In [12]:
for x in abstract_clean_l.values:
    one_doc = []
    for y in x:
        one_doc.append(dictionary.token2id[y])
        
    new_corpus.append(one_doc)

In [13]:
len(new_vocab)

51903

In [101]:
hlda = HierarchicalLDA(new_corpus, new_vocab, alpha=alpha, gamma=gamma, eta=eta, num_levels=num_levels)

In [102]:
hlda.estimate(n_samples, display_topics=display_topics, n_words=n_words, with_weights=with_weights)

HierarchicalLDA sampling
.................................................. 50
topic 0 (level=0, total_words=1018138, documents=37359): data, paper, model, approach, using, 
    topic 1 (level=1, total_words=293323, documents=12064): algorithm, time, network, show, results, 
        topic 2 (level=2, total_words=85947, documents=2813): problem, algorithm, model, learning, networks, 
        topic 6 (level=2, total_words=86362, documents=2190): graph, problem, graphs, algorithm, time, 
        topic 8 (level=2, total_words=605, documents=18): lumen, robot, speech, module, face, 
        topic 12 (level=2, total_words=104014, documents=3117): power, design, performance, memory, energy, 
        topic 17 (level=2, total_words=108918, documents=3210): data, system, software, systems, users, 
        topic 23 (level=2, total_words=515, documents=30): team, teams, sports, coach, competition, 
        topic 31 (level=2, total_words=2729, documents=103): cancer, genes, gene, patients, drug, 
 

        topic 76 (level=2, total_words=101, documents=11): theme, saccade, mias, subgraph, funny, 
        topic 87 (level=2, total_words=37, documents=3): stepgan, tent, seqgan, goodness, parasitic, 
        topic 89 (level=2, total_words=77, documents=7): itemsets, hins, infrequent, diffeomorphisms, rule, 
        topic 100 (level=2, total_words=116, documents=7): noisin, concessions, syriac, argumentative, minds, 
        topic 118 (level=2, total_words=20, documents=2): verifiers, original, boogie, link, insert, 
.................................................. 100
topic 0 (level=0, total_words=1012291, documents=37359): data, model, paper, approach, system, 
    topic 1 (level=1, total_words=298425, documents=12159): algorithm, time, show, results, paper, 
        topic 2 (level=2, total_words=77673, documents=2677): networks, model, problem, learning, network, 
        topic 6 (level=2, total_words=86277, documents=2325): graph, graphs, problem, algorithm, time, 
        topic 

.................................................. 150
topic 0 (level=0, total_words=1007545, documents=37359): data, paper, model, system, approach, 
    topic 1 (level=1, total_words=301868, documents=12223): algorithm, show, time, results, problem, 
        topic 2 (level=2, total_words=74769, documents=2572): networks, learning, network, model, problem, 
        topic 6 (level=2, total_words=87357, documents=2361): graph, graphs, problem, algorithm, time, 
        topic 8 (level=2, total_words=750, documents=22): lumen, module, robot, skull, speech, 
        topic 12 (level=2, total_words=93964, documents=2667): power, design, energy, memory, neural, 
        topic 17 (level=2, total_words=99810, documents=2878): data, software, system, systems, users, 
        topic 23 (level=2, total_words=1083, documents=54): team, game, teams, sports, ranking, 
        topic 31 (level=2, total_words=5881, documents=203): cancer, patients, clinical, medical, gene, 
        topic 32 (level=2, tot

.........

KeyboardInterrupt: 

[(0, 'able'),
 (1, 'accurate'),
 (2, 'action'),
 (3, 'adopted'),
 (4, 'also'),
 (5, 'analysis'),
 (6, 'answer'),
 (7, 'autonomous'),
 (8, 'based'),
 (9, 'baseline'),
 (10, 'behavior'),
 (11, 'benchmark'),
 (12, 'brier'),
 (13, 'caused'),
 (14, 'comparing'),
 (15, 'consequences'),
 (16, 'conservatism'),
 (17, 'construct'),
 (18, 'corresponding'),
 (19, 'criticality'),
 (20, 'defined'),
 (21, 'designated'),
 (22, 'different'),
 (23, 'discuss'),
 (24, 'driving'),
 (25, 'employ'),
 (26, 'entities'),
 (27, 'evaluated'),
 (28, 'evaluation'),
 (29, 'exemplar'),
 (30, 'existing'),
 (31, 'fatality'),
 (32, 'formulate'),
 (33, 'formulation'),
 (34, 'framework'),
 (35, 'future'),
 (36, 'generate'),
 (37, 'generated'),
 (38, 'graphical'),
 (39, 'highly'),
 (40, 'highway'),
 (41, 'homogenize'),
 (42, 'inaccurate'),
 (43, 'indicate'),
 (44, 'indicator'),
 (45, 'intention'),
 (46, 'interacting'),
 (47, 'interactive'),
 (48, 'inverse'),
 (49, 'learning'),
 (50, 'method'),
 (51, 'methods'),
 (52, 'metr

In [1]:
dictionary.id2token[3162286]

NameError: name 'dictionary' is not defined

In [14]:
# build skipgrams feature from sentences over the abstract
from nltk.util import skipgrams

In [20]:
def alphabetic_only_sentence(x_string):
    sentences = x_string.lower().split(".")
    all_sentence = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        # remove all tokens that are not alphabetic
        words = [word for word in tokens if word.isalpha() and word not in stop_words and len(word)>3]
        all_sentence.append(words)
    return all_sentence

In [22]:
test = alphabetic_only_sentence(dataset.iloc[0].abstract)

In [36]:
list(skipgrams(test[2],3,2))

[('existing', 'unified', 'framework'),
 ('existing', 'unified', 'homogenize'),
 ('existing', 'unified', 'problem'),
 ('existing', 'framework', 'homogenize'),
 ('existing', 'framework', 'problem'),
 ('existing', 'homogenize', 'problem'),
 ('unified', 'framework', 'homogenize'),
 ('unified', 'framework', 'problem'),
 ('unified', 'framework', 'formulation'),
 ('unified', 'homogenize', 'problem'),
 ('unified', 'homogenize', 'formulation'),
 ('unified', 'problem', 'formulation'),
 ('framework', 'homogenize', 'problem'),
 ('framework', 'homogenize', 'formulation'),
 ('framework', 'homogenize', 'representation'),
 ('framework', 'problem', 'formulation'),
 ('framework', 'problem', 'representation'),
 ('framework', 'formulation', 'representation'),
 ('homogenize', 'problem', 'formulation'),
 ('homogenize', 'problem', 'representation'),
 ('homogenize', 'problem', 'simplification'),
 ('homogenize', 'formulation', 'representation'),
 ('homogenize', 'formulation', 'simplification'),
 ('homogenize',

# using bag of words for classification

In [52]:
dictionary.from_corpus()

TypeError: from_corpus() missing 1 required positional argument: 'corpus'

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [60]:
cvector = CountVectorizer()

In [61]:
bow = cvector.fit_transform(abstract_clean)

In [62]:
# we have 51,903 features / words
bow.shape

(37359, 51903)

In [64]:
bow[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [77]:
bow[0,np.argwhere(bow[0]!=0)[:,1]].toarray()

array([[1, 1, 1, 1, 2, 1, 1, 2, 2, 3, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1,
        1, 2, 1, 2, 1, 1, 1, 3, 4, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 3, 5, 4, 1, 2, 1, 1, 2, 2, 1, 2, 1, 3, 1, 1, 1, 1, 2, 1, 2,
        1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2]],
      dtype=int64)

In [78]:
# make a neural net
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Bidirectional
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tflearn.data_utils import to_categorical, pad_sequences

Using TensorFlow backend.


In [90]:
label_class = dataset["class"].unique()
label_class

array(['cs.AI', 'cs.AR', 'cs.CC', 'cs.CE', 'cs.CG', 'cs.CL', 'cs.CR',
       'cs.CV', 'cs.CY', 'cs.DB', 'cs.DC', 'cs.DL', 'cs.DM', 'cs.DS',
       'cs.ET', 'cs.FL', 'cs.GL', 'cs.GR', 'cs.GT', 'cs.HC', 'cs.IR',
       'cs.IT', 'cs.LG', 'cs.LO', 'cs.MA', 'cs.MM', 'cs.MS', 'cs.NA',
       'cs.NE', 'cs.NI', 'cs.OH', 'cs.OS', 'cs.PF', 'cs.PL', 'cs.RO',
       'cs.SC', 'cs.SD', 'cs.SE', 'cs.SI', 'cs.SY'], dtype=object)

In [97]:
?np.argwhere(label_class=="cs.AI")

In [162]:
from keras.layers import Input
from keras.models import Model


model = Sequential()
# input layer
model.add(keras.layers.Dense(5000,input_shape=(bow.shape[1],),activation='relu'))

# hidden layer
model.add(keras.layers.Dense(1000,activation='relu'))
model.add(keras.layers.Dense(500,activation='relu'))
model.add(keras.layers.Dense(100,activation='relu'))

# output layer
model.add(keras.layers.Dense(len(label_class),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [163]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5000)              259520000 
_________________________________________________________________
dense_14 (Dense)             (None, 1000)              5001000   
_________________________________________________________________
dense_15 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_16 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_17 (Dense)             (None, 40)                4040      
Total params: 265,075,640
Trainable params: 265,075,640
Non-trainable params: 0
_________________________________________________________________


In [164]:
import json

def labeling_subjects(subjects):
    subs = json.loads(subjects)
    output = np.zeros(len(label_class),dtype=int)
    for sub in [np.argwhere(label_class==sub) for sub in subs]:
        if len(sub)>0:
            output[sub] = 1
    return output

In [165]:
test = np.array([])
for x in dataset.iloc[0:10].subjects:
    test = np.append(test,labeling_subjects(x),axis=0)

test = test.reshape(10,40)
test.shape
#dataset.iloc[0:10].subjects.apply(lambda x:)

(10, 40)

In [169]:
# processing the model

import random

mini_batch = 1000
total_docs = len(abstract_clean_l)
total_batch = int(len(abstract_clean_l)/mini_batch)+1
range_docs = range(0,total_docs)


for batch in range(total_batch):
    # random sample dataset
    r_index = random.sample(range_docs,mini_batch)
    X_batch = bow[r_index].toarray()
    #y_batch = dataset.iloc[r_index].subjects.apply(lambda x:labeling_subjects(x))
    y_batch = np.array([])
    for x in dataset.iloc[r_index].subjects:
        y_batch = np.append(y_batch,labeling_subjects(x),axis=0)

    y_batch = y_batch.reshape(len(r_index),len(label_class))
    model.fit(X_batch,y_batch,epochs=1,batch_size=200)


Epoch 1/1
1000/1000 [==============================] - 67s 67ms/step - loss: 5.0542 - acc: 0.0530
Epoch 1/1
1000/1000 [==============================] - 53s 53ms/step - loss: 4.5028 - acc: 0.1340
Epoch 1/1
1000/1000 [==============================] - 46s 46ms/step - loss: 3.9054 - acc: 0.2620
Epoch 1/1
1000/1000 [==============================] - 46s 46ms/step - loss: 3.3980 - acc: 0.3230
Epoch 1/1
1000/1000 [==============================] - 44s 44ms/step - loss: 3.1560 - acc: 0.4120
Epoch 1/1
1000/1000 [==============================] - 50s 50ms/step - loss: 2.9297 - acc: 0.4790
Epoch 1/1
1000/1000 [==============================] - 46s 46ms/step - loss: 2.5443 - acc: 0.5290
Epoch 1/1
1000/1000 [==============================] - 43s 43ms/step - loss: 2.6392 - acc: 0.5270
Epoch 1/1
1000/1000 [==============================] - 40s 40ms/step - loss: 2.6154 - acc: 0.5320
Epoch 1/1
1000/1000 [==============================] - 41s 41ms/step - loss: 2.4679 - acc: 0.5260
Epoch 1/1
1000/1000 

In [170]:
# save model
model.save_weights('model-keras-bow-01.h5')    

In [171]:
for batch in range(total_batch):
    # random sample dataset
    r_index = random.sample(range_docs,mini_batch)
    X_batch = bow[r_index].toarray()
    #y_batch = dataset.iloc[r_index].subjects.apply(lambda x:labeling_subjects(x))
    y_batch = np.array([])
    for x in dataset.iloc[r_index].subjects:
        y_batch = np.append(y_batch,labeling_subjects(x),axis=0)

    y_batch = y_batch.reshape(len(r_index),len(label_class))
    model.fit(X_batch,y_batch,epochs=1,batch_size=200)

Epoch 1/1
1000/1000 [==============================] - 56s 56ms/step - loss: 1.6598 - acc: 0.7150
Epoch 1/1
1000/1000 [==============================] - 43s 43ms/step - loss: 1.6303 - acc: 0.6910
Epoch 1/1
1000/1000 [==============================] - 44s 44ms/step - loss: 1.6704 - acc: 0.6920
Epoch 1/1
1000/1000 [==============================] - 44s 44ms/step - loss: 1.5685 - acc: 0.7070
Epoch 1/1
1000/1000 [==============================] - 41s 41ms/step - loss: 1.6707 - acc: 0.7090
Epoch 1/1
1000/1000 [==============================] - 46s 46ms/step - loss: 1.5935 - acc: 0.7110
Epoch 1/1
1000/1000 [==============================] - 46s 46ms/step - loss: 1.6225 - acc: 0.6740
Epoch 1/1
1000/1000 [==============================] - 52s 52ms/step - loss: 1.6219 - acc: 0.7130
Epoch 1/1
1000/1000 [==============================] - 47s 47ms/step - loss: 1.5140 - acc: 0.7400
Epoch 1/1
1000/1000 [==============================] - 41s 41ms/step - loss: 1.6930 - acc: 0.7100
Epoch 1/1
1000/1000 

In [172]:
model.save_weights('model-keras-bow-02.h5')    